In [1]:
# imports
import os
os.sys.path.append(os.path.abspath(".."))

import numpy as np
import pandas as pd

import h5py

import project.download_content as content
from project.utils import data

from tqdm import tqdm

import pickle

modelpath = os.path.join(content.DATAPATH, "MODEL")

tqdm.pandas()

This notebook is about create the environment to start to train models.

## About boats?!

To early exploration in modeling I chosen to work with boats bbox. They are the 30th most relevant class, this means that they are big enough to be isolated, but they are small enough to start a model exploration (stress cost functions, architectures, so on).

In [2]:
#reading all train csvs
all_train = pd.concat(
    [pd.read_csv(os.path.join(modelpath, f'train_preprocessed_{i}.csv')) for i in range(8)]
)
all_train['bbox_ref'] = all_train.bbox_ref.apply(lambda x: [int(k) for k 
                                                            in filter(lambda a: a, x[1:-1].split(' '))])
all_train['bbox_count'] = all_train.bbox_ref.apply(lambda x: len(x))
all_train.head(5)

,ImageID,LabelName,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,Path,LabelSemantic,cx,cy,w,h,boat,land_vehicle,skyscraper,bbox_ref,bbox_count
0,00001bc2c4027449,/m/019jd,1,0,0,0,0,data/TRAIN/train_0/00001bc2c4027449.jpg,boat,0.516250,0.521667,0.497500,0.561667,True,False,False,[],0
1,0000339d0372e7e6,/m/079cl,1,0,0,0,0,data/TRAIN/train_0/0000339d0372e7e6.jpg,skyscraper,0.382188,0.687675,0.056875,0.126050,False,False,True,[],0
2,0000339d0372e7e6,/m/079cl,1,0,0,0,0,data/TRAIN/train_0/0000339d0372e7e6.jpg,skyscraper,0.382188,0.683474,0.029375,0.102707,False,False,True,[],0
3,0000339d0372e7e6,/m/079cl,1,1,1,0,0,data/TRAIN/train_0/0000339d0372e7e6.jpg,skyscraper,0.519062,0.782446,0.174375,0.330532,False,False,True,"[8712, 8713]",2
4,0000339d0372e7e6,/m/079cl,1,0,0,0,0,data/TRAIN/train_0/0000339d0372e7e6.jpg,skyscraper,0.516563,0.790850,0.079375,0.229692,False,False,True,[],0


In [3]:
train = (pd.concat([all_train.query("bbox_count > 0")])
           .sample(frac=1, random_state=17))
display(train.head(5))
train.shape

,ImageID,LabelName,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,Path,LabelSemantic,cx,cy,w,h,boat,land_vehicle,skyscraper,bbox_ref,bbox_count
2336,5f12df8a5b56bcb1,/m/079cl,1,0,0,0,0,data/TRAIN/train_5/5f12df8a5b56bcb1.jpg,skyscraper,0.713437,0.500417,0.100625,0.079167,False,False,True,"[8644, 8647]",2
23685,f8a779759756a936,/m/019jd,1,1,0,0,0,data/TRAIN/train_f/f8a779759756a936.jpg,boat,0.469375,0.485459,0.916250,0.663227,True,False,False,"[8728, 8730, 8731]",3
20961,9464222fcbb5f608,/m/01prls,1,1,0,0,0,data/TRAIN/train_9/9464222fcbb5f608.jpg,land_vehicle,0.838816,0.437812,0.320489,0.295625,False,True,False,"[8720, 8722, 8723]",3
28793,3a9423a2a5d3c594,/m/079cl,1,0,0,0,0,data/TRAIN/train_3/3a9423a2a5d3c594.jpg,skyscraper,0.845625,0.648333,0.030000,0.105000,False,False,True,"[8459, 8460]",2
9809,c87b9e81c490a73e,/m/019jd,0,0,0,0,0,data/TRAIN/train_c/c87b9e81c490a73e.jpg,boat,0.645625,0.535481,0.043750,0.047619,True,False,False,[8333],1


(36480, 18)

creating the default bounding boxes:

In [4]:
standard_bboxes = data.StandardBoudingBoxes(feature_map_sizes=[38, 19, 10, 5, 3, 1],
                                            ratios_per_layer=[[1, 1/2, 2], 
                                                              [1, 1/2, 1/3, 2, 3],
                                                              [1, 1/2, 1/3, 2, 3],
                                                              [1, 1/2, 1/3, 2, 3],
                                                              [1, 1/2, 2],
                                                              [1, 1/2, 2]])

standard_bboxes.references.values

array([[0.01315789, 0.01315789, 0.00526316, 0.00526316],
       [0.01315789, 0.01315789, 0.00372161, 0.00744323],
       [0.01315789, 0.01315789, 0.00744323, 0.00372161],
       ...,
       [0.5       , 0.5       , 0.6363961 , 1.27279221],
       [0.5       , 0.5       , 1.27279221, 0.6363961 ],
       [0.5       , 0.5       , 0.96747093, 0.96747093]])

In [5]:
%%time
imgs = train[['ImageID', 'Path']].drop_duplicates()
#getting the one hot encodes columns
dummy_classes = list(train.columns[13:-2])

imgs.set_index('ImageID', inplace=True)

for c in ['cx', 'cy', 'w', 'h', 'bbox_ref', 'LabelSemantic'] + dummy_classes:
    imgs = imgs.join(train.groupby('ImageID')[c].apply(list))

imgs.reset_index(inplace=True)
display(imgs.shape)
imgs.head()

(31944, 11)

CPU times: user 14.8 s, sys: 36.3 ms, total: 14.9 s
Wall time: 14.9 s


,ImageID,Path,cx,cy,w,h,bbox_ref,LabelSemantic,boat,land_vehicle,skyscraper
0,5f12df8a5b56bcb1,data/TRAIN/train_5/5f12df8a5b56bcb1.jpg,[0.7134375],[0.5004165],[0.10062500000000008],[0.07916700000000004],"[[8644, 8647]]",[skyscraper],[False],[False],[True]
1,f8a779759756a936,data/TRAIN/train_f/f8a779759756a936.jpg,[0.469375],[0.4854594999999999],[0.91625],[0.6632269999999999],"[[8728, 8730, 8731]]",[boat],[True],[False],[False]
2,9464222fcbb5f608,data/TRAIN/train_9/9464222fcbb5f608.jpg,[0.8388155],[0.4378125],[0.320489],[0.295625],"[[8720, 8722, 8723]]",[land_vehicle],[False],[True],[False]
3,3a9423a2a5d3c594,data/TRAIN/train_3/3a9423a2a5d3c594.jpg,[0.845625],[0.648333],[0.030000000000000027],[0.1050000000000001],"[[8459, 8460]]",[skyscraper],[False],[False],[True]
4,c87b9e81c490a73e,data/TRAIN/train_c/c87b9e81c490a73e.jpg,[0.645625],[0.5354805],[0.04374999999999996],[0.04761899999999997],[[8333]],[boat],[True],[False],[False]


At this point, we are creating the target dataset

applying the transformations explain in page 5 of the [ssd paper](https://arxiv.org/pdf/1512.02325.pdf)

In [6]:
# This 6 is because the output of the model for each bbox found is 6: 2 classes + (cx, cy, w, h)
target = np.zeros((imgs.shape[0],
                   standard_bboxes.references.shape[0],
                   (1 + len(dummy_classes) + 4)),
                  dtype=np.float32)
target[:][:] = [1] + [0] * len(dummy_classes) + [0,0,0,0]

for i, row in imgs.iterrows():
    for cx, cy, w, h, refs, *labels in zip(row.cx, row.cy, row.w, row.h, row.bbox_ref, *row[8:]):
        for id_ref in refs:
            target[i][int(id_ref)] = [0] + labels + [cx, cy, w, h]
        
target.shape

(31944, 8732, 8)

In [7]:
for i, r in enumerate(target[0]):
    if r[1] == 1:
        print(1, i, r[-4], r[-3], r[-2], r[-1])
    if r[2] == 1:
        print(2, i)
    if r[3] == 1:
        print(3, i)

3 8644
3 8647


### Hdf5

In [8]:
filepath = os.path.join(content.DATAPATH, "MODEL", "part_data_300_vgg.h5")

In [9]:
imgs.to_hdf(filepath, key='X', mode='w')

with h5py.File(filepath, 'a') as f:
    f.create_dataset('y', data=target, dtype=np.float16)

#### Reading

In [10]:
read_data = pd.read_hdf(filepath, 'X', mode='r')

In [11]:
with h5py.File(filepath, 'r') as f:
    dset = f['y'][:]
dset.shape

(31944, 8732, 8)

In [12]:
read_data.head(4)

,ImageID,Path,cx,cy,w,h,bbox_ref,LabelSemantic,boat,land_vehicle,skyscraper
0,5f12df8a5b56bcb1,data/TRAIN/train_5/5f12df8a5b56bcb1.jpg,[0.7134375],[0.5004165],[0.10062500000000008],[0.07916700000000004],"[[8644, 8647]]",[skyscraper],[False],[False],[True]
1,f8a779759756a936,data/TRAIN/train_f/f8a779759756a936.jpg,[0.469375],[0.4854594999999999],[0.91625],[0.6632269999999999],"[[8728, 8730, 8731]]",[boat],[True],[False],[False]
2,9464222fcbb5f608,data/TRAIN/train_9/9464222fcbb5f608.jpg,[0.8388155],[0.4378125],[0.320489],[0.295625],"[[8720, 8722, 8723]]",[land_vehicle],[False],[True],[False]
3,3a9423a2a5d3c594,data/TRAIN/train_3/3a9423a2a5d3c594.jpg,[0.845625],[0.648333],[0.030000000000000027],[0.1050000000000001],"[[8459, 8460]]",[skyscraper],[False],[False],[True]


In [13]:
standard_bboxes.references.iloc[8713]

cx    0.500000
cy    0.833333
w     0.179134
h     0.358267
Name: 8713, dtype: float64

In [14]:
dset[0][8644]

array([0.     , 0.     , 0.     , 1.     , 0.7134 , 0.5005 , 0.10065,
       0.07916], dtype=float16)